In [5]:
import numpy as np
from skimage.feature import graycomatrix, graycoprops
from skimage import io, color, img_as_ubyte

def contrast_feature(matrix_coocurrence):
	contrast = graycoprops(matrix_coocurrence, 'contrast')
	return contrast

def dissimilarity_feature(matrix_coocurrence):
	dissimilarity = graycoprops(matrix_coocurrence, 'dissimilarity')	
	return dissimilarity

def homogeneity_feature(matrix_coocurrence):
	homogeneity = graycoprops(matrix_coocurrence, 'homogeneity')
	return homogeneity

def energy_feature(matrix_coocurrence):
	energy = graycoprops(matrix_coocurrence, 'energy')
	return energy

def correlation_feature(matrix_coocurrence):
	correlation = graycoprops(matrix_coocurrence, 'correlation')
	return correlation

def asm_feature(matrix_coocurrence):
	asm = graycoprops(matrix_coocurrence, 'ASM')
	return asm

from scipy.signal import savgol_filter
def ind_gen(entropy,len1):
    #smoothing
    yhat = savgol_filter(entropy,3,2)
    #미분
    diff = []
    for i in range(len(yhat)-1):
        diff.append(abs(yhat[i]-yhat[i+1])) 
    diff = np.array(diff)    
    diff = diff / diff.sum() 
    #cdf
    cdf = []
    for i in range(len(diff)):
        if i == 0:
            cdf.append(diff[i])
        else :     
            cdf.append(diff[i]+cdf[-1])
    #ind찾기
    a = cdf
    n_total = [1/15,2/15,3/15,4/15,5/15,6/15,7/15,8/15,9/15,10/15,11/15,12/15,13/15,14/15]
    ind = [0]
    for j in range(len(n_total)):
        b = []
        n = n_total[j]
        for i in a:
            tmp =abs(i-n)
            b.append(tmp)
            #tmp =abs(i-n)


        b.sort()
        search_val = b[0]


        for i in a:
            tmp = abs(i-n)
            if tmp == search_val:
                ind_temp = cdf.index(i)
                ind.append(ind_temp)
                break

    ind.append(len1-1)
    return ind

In [6]:
import cv2
import os
import glob
import math
import numpy as np
import natsort
from scipy.signal import savgol_filter
set1 = "/data/kdu/covid_eccv_pc5_original/test/set1"
set2 = "/data/kdu/covid_eccv_pc5_original/test/set2"
set3 = "/data/kdu/covid_eccv_pc5_original/test/set3"
set4 = "/data/kdu/covid_eccv_pc5_original/test/set4"
set5 = "/data/kdu/covid_eccv_pc5_original/test/set5"
set6 = "/data/kdu/covid_eccv_pc5_original/test/set6"

set1_list = glob.glob(os.path.join(set1,'ct*'))
set2_list = glob.glob(os.path.join(set2,'ct*'))
set3_list = glob.glob(os.path.join(set3,'ct*'))
set4_list = glob.glob(os.path.join(set4,'ct*'))
set5_list = glob.glob(os.path.join(set5,'ct*'))
set6_list = glob.glob(os.path.join(set6,'ct*'))

set1_list = natsort.natsorted(set1_list)
set2_list = natsort.natsorted(set2_list)
set3_list = natsort.natsorted(set3_list)
set4_list = natsort.natsorted(set4_list)
set5_list = natsort.natsorted(set5_list)
set6_list = natsort.natsorted(set6_list)

list_list = [set1_list,set2_list,set3_list,set4_list,set5_list,set6_list]
for file_list in list_list:
    for fol in file_list:
        file_list = glob.glob(os.path.join(fol,"*.jpg"))
        file_list = natsort.natsorted(file_list)
        entropy = []


        len1 = len(file_list)
        if len1 < 17:
            for idx in range(len1):
                gray = cv2.imread(file_list[idx])
                # save directory check
                save_path = file_list[idx].replace('covid_eccv_pc5_original','covid_eccv_pc5_original_adapsam_3')
                #save_path = img_path_list[idx].replace('covid_eccv','covid_eccv_pc5_original')
                save_folder, _ = os.path.split(save_path)
                if not os.path.exists(save_folder):
                    os.makedirs(save_folder)

                cv2.imwrite(save_path, gray.astype('uint8'))

        else :    

            for gray_path in file_list:
                gray = cv2.imread(gray_path)/255
                gray = color.rgb2gray(gray)
                image = img_as_ubyte(gray)
                #bins = np.array([0, 16, 32, 48, 64, 80, 96, 112, 128, 144, 160, 176, 192, 208, 224, 240, 255]) #16-bit
               # bins = np.array([0, 32, 64, 96, 128, 160, 192, 224, 255]) #8-bit
                bins = np.array([0, 8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96, 104, 112, 120, 128, 136, 144, 152, 160, 168, 176, 184, 192, 200, 208, 216, 224, 232, 240, 248, 255]) #32-bit
                inds = np.digitize(image, bins)

                max_value = inds.max()+1


                matrix_coocurrence = graycomatrix(inds, [1],  [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=max_value, normed=True, symmetric=False)
                ent_total = -1*matrix_coocurrence*(np.log(matrix_coocurrence+0.000001))
                entropy.append(ent_total[:,:,:,:].sum())
            index = ind_gen(entropy, len1)

    #         print(index)
    #         interval = int(len1 / 16)
    #         ori = []
    #         for k in range(16):
    #             ori.append(k*interval)
    #         print(ori)   
           # print(index)
          #  print(len1)
            for ind in index:
                gray = cv2.imread(file_list[ind])
                # save directory check
                save_path = file_list[ind].replace('covid_eccv_pc5_original','covid_eccv_pc5_original_adapsam_3')
                #save_path = img_path_list[idx].replace('covid_eccv','covid_eccv_pc5_original')
                save_folder, _ = os.path.split(save_path)
                if not os.path.exists(save_folder):
                    os.makedirs(save_folder)
               # print(save_path)
                cv2.imwrite(save_path, gray.astype('uint8'))
            
            
                
            



In [1]:
import cv2
import os
import glob
import math
import numpy as np
import natsort
from scipy.signal import savgol_filter
set1 = "/data/kdu/covid_eccv_pc5_original/test/set1"
set2 = "/data/kdu/covid_eccv_pc5_original/test/set2"
set3 = "/data/kdu/covid_eccv_pc5_original/test/set3"
set4 = "/data/kdu/covid_eccv_pc5_original/test/set4"
set5 = "/data/kdu/covid_eccv_pc5_original/test/set5"
set6 = "/data/kdu/covid_eccv_pc5_original/test/set6"

set1_list = glob.glob(os.path.join(set1,'ct*'))
set2_list = glob.glob(os.path.join(set2,'ct*'))
set3_list = glob.glob(os.path.join(set3,'ct*'))
set4_list = glob.glob(os.path.join(set4,'ct*'))
set5_list = glob.glob(os.path.join(set5,'ct*'))
set6_list = glob.glob(os.path.join(set6,'ct*'))

set1_list = natsort.natsorted(set1_list)
set2_list = natsort.natsorted(set2_list)
set3_list = natsort.natsorted(set3_list)
set4_list = natsort.natsorted(set4_list)
set5_list = natsort.natsorted(set5_list)
set6_list = natsort.natsorted(set6_list)

list_list = [set1_list,set2_list,set3_list,set4_list,set5_list,set6_list]
for file_list in list_list:
    for fol in file_list:
        file_list = glob.glob(os.path.join(fol,"*.jpg"))
        file_list = natsort.natsorted(file_list)
        entropy = []


        len1 = len(file_list)
        if len1 < 17:
            for idx in range(len1):
                gray = cv2.imread(file_list[idx])
                # save directory check
                save_path = file_list[idx].replace('covid_eccv_pc5_original','covid_eccv_pc5_original_adapsam_9')
                #save_path = img_path_list[idx].replace('covid_eccv','covid_eccv_pc5_original')
                save_folder, _ = os.path.split(save_path)
                if not os.path.exists(save_folder):
                    os.makedirs(save_folder)

                cv2.imwrite(save_path, gray.astype('uint8'))

        else :    

            for gray_path in file_list:
                gray = cv2.imread(gray_path)/255
                gray = color.rgb2gray(gray)
                image = img_as_ubyte(gray)
                #bins = np.array([0, 16, 32, 48, 64, 80, 96, 112, 128, 144, 160, 176, 192, 208, 224, 240, 255]) #16-bit
               # bins = np.array([0, 32, 64, 96, 128, 160, 192, 224, 255]) #8-bit
                bins = np.array([0, 8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96, 104, 112, 120, 128, 136, 144, 152, 160, 168, 176, 184, 192, 200, 208, 216, 224, 232, 240, 248, 255]) #32-bit
                inds = np.digitize(image, bins)

                max_value = inds.max()+1


                matrix_coocurrence = graycomatrix(inds, [1],  [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=max_value, normed=True, symmetric=False)
                ent_total = -1*matrix_coocurrence*(np.log(matrix_coocurrence+0.000001))
                entropy.append(ent_total[:,:,:,:].sum())
            index = ind_gen(entropy, len1)

    #         print(index)
    #         interval = int(len1 / 16)
    #         ori = []
    #         for k in range(16):
    #             ori.append(k*interval)
    #         print(ori)   
           # print(index)
          #  print(len1)
            for ind in index:
                gray = cv2.imread(file_list[ind])
                # save directory check
                save_path = file_list[ind].replace('covid_eccv_pc5_original','covid_eccv_pc5_original_adapsam_9')
                #save_path = img_path_list[idx].replace('covid_eccv','covid_eccv_pc5_original')
                save_folder, _ = os.path.split(save_path)
                if not os.path.exists(save_folder):
                    os.makedirs(save_folder)
               # print(save_path)
                cv2.imwrite(save_path, gray.astype('uint8'))
            
            
                
            



In [4]:
import cv2
import os
import glob
import math
import numpy as np
import natsort
from scipy.signal import savgol_filter
set1 = "/data/kdu/covid_eccv_pc5_original/test/set1"
set2 = "/data/kdu/covid_eccv_pc5_original/test/set2"
set3 = "/data/kdu/covid_eccv_pc5_original/test/set3"
set4 = "/data/kdu/covid_eccv_pc5_original/test/set4"
set5 = "/data/kdu/covid_eccv_pc5_original/test/set5"
set6 = "/data/kdu/covid_eccv_pc5_original/test/set6"

set1_list = glob.glob(os.path.join(set1,'ct*'))
set2_list = glob.glob(os.path.join(set2,'ct*'))
set3_list = glob.glob(os.path.join(set3,'ct*'))
set4_list = glob.glob(os.path.join(set4,'ct*'))
set5_list = glob.glob(os.path.join(set5,'ct*'))
set6_list = glob.glob(os.path.join(set6,'ct*'))

set1_list = natsort.natsorted(set1_list)
set2_list = natsort.natsorted(set2_list)
set3_list = natsort.natsorted(set3_list)
set4_list = natsort.natsorted(set4_list)
set5_list = natsort.natsorted(set5_list)
set6_list = natsort.natsorted(set6_list)

list_list = [set1_list,set2_list,set3_list,set4_list,set5_list,set6_list]
for file_list in list_list:
    for fol in file_list:
        file_list = glob.glob(os.path.join(fol,"*.jpg"))
        file_list = natsort.natsorted(file_list)
        entropy = []


        len1 = len(file_list)
        if len1 < 17:
            for idx in range(len1):
                gray = cv2.imread(file_list[idx])
                # save directory check
                save_path = file_list[idx].replace('covid_eccv_pc5_original','covid_eccv_pc5_original_adapsam_17')
                #save_path = img_path_list[idx].replace('covid_eccv','covid_eccv_pc5_original')
                save_folder, _ = os.path.split(save_path)
                if not os.path.exists(save_folder):
                    os.makedirs(save_folder)

                cv2.imwrite(save_path, gray.astype('uint8'))

        else :    

            for gray_path in file_list:
                gray = cv2.imread(gray_path)/255
                gray = color.rgb2gray(gray)
                image = img_as_ubyte(gray)
                #bins = np.array([0, 16, 32, 48, 64, 80, 96, 112, 128, 144, 160, 176, 192, 208, 224, 240, 255]) #16-bit
               # bins = np.array([0, 32, 64, 96, 128, 160, 192, 224, 255]) #8-bit
                bins = np.array([0, 8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96, 104, 112, 120, 128, 136, 144, 152, 160, 168, 176, 184, 192, 200, 208, 216, 224, 232, 240, 248, 255]) #32-bit
                inds = np.digitize(image, bins)

                max_value = inds.max()+1


                matrix_coocurrence = graycomatrix(inds, [1],  [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=max_value, normed=True, symmetric=False)
                ent_total = -1*matrix_coocurrence*(np.log(matrix_coocurrence+0.000001))
                entropy.append(ent_total[:,:,:,:].sum())
            index = ind_gen(entropy, len1)

    #         print(index)
    #         interval = int(len1 / 16)
    #         ori = []
    #         for k in range(16):
    #             ori.append(k*interval)
    #         print(ori)   
           # print(index)
          #  print(len1)
            for ind in index:
                gray = cv2.imread(file_list[ind])
                # save directory check
                save_path = file_list[ind].replace('covid_eccv_pc5_original','covid_eccv_pc5_original_adapsam_17')
                #save_path = img_path_list[idx].replace('covid_eccv','covid_eccv_pc5_original')
                save_folder, _ = os.path.split(save_path)
                if not os.path.exists(save_folder):
                    os.makedirs(save_folder)
               # print(save_path)
                cv2.imwrite(save_path, gray.astype('uint8'))
            
            
                
            



['/data/kdu/covid_eccv_pc5_original/test/set1/ct_scan_0', '/data/kdu/covid_eccv_pc5_original/test/set1/ct_scan_1', '/data/kdu/covid_eccv_pc5_original/test/set1/ct_scan_2', '/data/kdu/covid_eccv_pc5_original/test/set1/ct_scan_3', '/data/kdu/covid_eccv_pc5_original/test/set1/ct_scan_4', '/data/kdu/covid_eccv_pc5_original/test/set1/ct_scan_5', '/data/kdu/covid_eccv_pc5_original/test/set1/ct_scan_6', '/data/kdu/covid_eccv_pc5_original/test/set1/ct_scan_7', '/data/kdu/covid_eccv_pc5_original/test/set1/ct_scan_8', '/data/kdu/covid_eccv_pc5_original/test/set1/ct_scan_9', '/data/kdu/covid_eccv_pc5_original/test/set1/ct_scan_10', '/data/kdu/covid_eccv_pc5_original/test/set1/ct_scan_11', '/data/kdu/covid_eccv_pc5_original/test/set1/ct_scan_12', '/data/kdu/covid_eccv_pc5_original/test/set1/ct_scan_13', '/data/kdu/covid_eccv_pc5_original/test/set1/ct_scan_14', '/data/kdu/covid_eccv_pc5_original/test/set1/ct_scan_15', '/data/kdu/covid_eccv_pc5_original/test/set1/ct_scan_16', '/data/kdu/covid_eccv_p